# ECON 323 Final Project: Predicting Stress Levels through Sleep Data

## Introduction
In this project, we will aim to predict prices of airbnbs in NYC. As suggested in our project feedback, we will be making minimal use of existing package models. We hope to further explore the space of Machine Learning by instead coding our own models and using methods beyond the scope of this course. We were inspired to do price predictions as one of us was recently looking at airbnbs in NYC for a trip in the summer and surprisingly there was a dataset on this exact information. **Note: we pivoted from our inital sleep study test because after further investigation it seems it was an auto-generated dataset with no interesting findings.**

Below is a high-level outline of our procedure
1. Pre-process the data
2. Do some statstical analysis on features to see if there are any outliers and adjust accordingly
3. Reserve 20% of the dataset only to be used for testing once at the end
4. Use bootstraping to improve training phase
5. Build a random forest from scratch incrementally and display results throughout.
6. Present our results and interpretations numerically/visually (eg. training visualizations). 
7. Compare with common models from popular packages
7. Discussion about the shortcomings of our model and/or experimentation process.

#### Packages

In [297]:
! pip install patsy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing, ensemble, linear_model, utils

%matplotlib inline


[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Dataset

We will be utilizing a dataset with [New York City Airbnb Data](https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data?select=AB_NYC_2019.csv) as seen below. We will first do some preprocessing of the data to make it model compatible such as removing some columns and categorizing our target label "price". We will be categorizing it to approach it with classification methods and because price categories are intuitively reasonable. Then we will shuffle and seperate the dataset to our test and training sets.

Acknowledgements
1. L. Rachakonda, A. K. Bapatla, S. P. Mohanty, and E. Kougianos, “SaYoPillow: Blockchain-Integrated Privacy-Assured IoMT Framework for Stress Management Considering Sleeping Habits”, IEEE Transactions on Consumer Electronics (TCE), Vol. 67, No. 1, Feb 2021, pp. 20-29.
2. L. Rachakonda, S. P. Mohanty, E. Kougianos, K. Karunakaran, and M. Ganapathiraju, “Smart-Pillow: An IoT based Device for Stress Detection Considering Sleeping Habits”, in Proceedings of the 4th IEEE International Symposium on Smart Electronic Systems (iSES), 2018, pp. 161--166.

In [298]:
df = pd.read_csv("AB_NYC_2019.csv")
df = df[df["neighbourhood_group"] == "Manhattan"]
df = df[(df["price"] >= 100) & (df["price"] <= 700)]
print(len(df))
print(np.mean(df["price"]))
df = df.sample(n=5000) # only take 2000 otherwise takes too long
df = df.drop(["id", "name" , "host_id", "host_name", "last_review"], axis=1)
df = df.fillna(0)

# categorize price ranges
# bins_25 = np.array([i*50 for i in range(0,11)]) # bins of 25 from 0 to 500
# bins_50 = np.array([i*100 for i in range(6,16)]) # bins of 100 from 500 to 2000
# bins_100 = np.array([i*250 for i in range(7,41)]) # bins of 250 from 2000 to 10000
# bins = np.concatenate((bins_25, bins_50, bins_100))

# bins_25 = np.array([i*25 for i in range(0,21)]) # bins of 25 from 0 to 500
# bins_50 = np.array([i*50 for i in range(11,41)]) # bins of 100 from 500 to 2000
# bins_100 = np.array([i*100 for i in range(21,101)]) # bins of 250 from 2000 to 10000
# bins = np.concatenate((bins_25, bins_50, bins_100))

bins = [100,300,500,700]
print(bins)
labels = np.array(range(len(bins)-1))
class_count = len(labels)
df['price'] = pd.cut(df['price'], bins=bins, labels=labels)

# give labels to string fields
le = preprocessing.LabelEncoder()
le.fit(df["neighbourhood_group"])
df["neighbourhood_group"] = le.transform(df["neighbourhood_group"])
le.fit(df["neighbourhood"])
df["neighbourhood"] = le.transform(df["neighbourhood"])
le.fit(df["room_type"])
df["room_type"] = le.transform(df["room_type"])
le.fit(df["room_type"])
df["room_type"] = le.transform(df["room_type"])
le.fit(df["price"])
df["price"] = le.transform(df["price"])

print(df.head())
print(len(df))

num_training = int(len(df)*0.8)
X = df.drop(["price"], axis=1)
y = df["price"]
X, y = utils.shuffle(X, y, random_state=42)
n, d = X.shape
X_train = X.iloc[:num_training,:].values
X_test = X.iloc[num_training:,:].values
y_train = y.iloc[:num_training].values
y_test = y.iloc[num_training:].values



15184
209.55677028451
[100, 300, 500, 700]
       neighbourhood_group  neighbourhood  latitude  longitude  room_type  \
15058                    0             17  40.76647  -73.98290          0   
30000                    0              5  40.72939  -73.98553          0   
22700                    0             29  40.76989  -73.98229          0   
2170                     0             29  40.79962  -73.96523          1   
13326                    0              6  40.70623  -74.00906          1   

       price  minimum_nights  number_of_reviews  reviews_per_month  \
15058      1              30                  0               0.00   
30000      3               4                  5               0.30   
22700      0              30                  4               0.16   
2170       0               7                 70               0.93   
13326      3               1                  0               0.00   

       calculated_host_listings_count  availability_365  
15058          

### Statistical Analysis
We will cull any data that seems to be out of a reasonable range here

We will now bootstrap the training data, during training we will train on each sampled bootstrap and take the mode of the predictions.

In [299]:
num_bootstraps = 5
bootstrap_samples = np.zeros((num_bootstraps, num_training, X.shape[1]))
bootstrap_labels = np.zeros((num_bootstraps, num_training)) 
for i in range(num_bootstraps):
    # randomly sample the dataset with replacement
    bootstrap_indices = np.random.choice(num_training, num_training, replace=True)
    bootstrap_samples[i] = X_train[bootstrap_indices]
    bootstrap_labels[i] = y_train[bootstrap_indices]

## Creating the Decision Stump
We will use the most common splitting criterion of information gain which aims to select the feature that reduces entropy. This will be the most atomic unit that we construct our Decision Tree from.

In [300]:
from scipy.stats import entropy, mode

class DecisionStump:
    y_hat_yes = None
    y_hat_no = None
    j_best = None
    t_best = None

    def getProbs(self, n, classCount, count):
        if(n == 0): return []
        p = np.zeros(classCount)
        for i in range(classCount):
            p[i] = count[i]/n

        return p
    
    def mode(self, y_subset):
        if(len(y_subset) == 0): return -1
        return np.argmax(np.bincount(y_subset))

    def fit(self, X, y, feature_space=None):
        n, d = X.shape

        # Get an array with the number of 0's, number of 1's, etc.
        count = np.bincount(y, minlength=class_count)

        # Get the index of the largest value in count.
        # Thus, y_mode is the mode (most popular value) of y
        y_mode = np.argmax(count)

        self.y_hat_yes = y_mode

        # If all the labels are the same, no need to split further
        if np.unique(y).size <= 1:
            return

        # Get current entropy
        p = self.getProbs(n, class_count, count)
        ent = entropy(p)
        best_info_gain = 0
        
        if(feature_space is None):
            feature_space = range(d)

        # Loop over features looking for the best split
        for j in feature_space:
            for i in range(n):
                # Choose value for threshold 
                t = X[i,j] 

                yes_subset = y[X[:,j] > t]
                no_subset = y[X[:,j] <= t]

                # find best prediction for each side of the split
                y_yes_mode = self.mode(yes_subset)
                y_no_mode = self.mode(no_subset)

                # compute the entropy of each side of the split
                yes_counts = np.bincount(yes_subset, minlength=class_count)
                no_counts = np.bincount(no_subset, minlength=class_count)
                yes_p = self.getProbs(len(yes_subset), class_count, yes_counts)
                no_p = self.getProbs(len(no_subset), class_count, no_counts)

                ent_yes = (len(yes_subset)/n)*entropy(yes_p) 
                ent_no = (len(no_subset)/n)*entropy(no_p)
                weighted_entropy = ent_yes + ent_no

                # calculate info gain from the splits when weighted
                information_gain = ent - weighted_entropy

                # Compare to information gain so far
                if information_gain > best_info_gain:
                    best_info_gain = information_gain
                    self.j_best = j
                    self.t_best = t
                    self.y_hat_yes = y_yes_mode
                    self.y_hat_no = y_no_mode   



    def predict(self, X):
        n, d = X.shape

        if self.j_best is None:
            return self.y_hat_yes * np.ones(n)

        y_hat = np.zeros(n)

        # assign predictions
        for i in range(n):
            if X[i, self.j_best] > self.t_best:
                y_hat[i] = self.y_hat_yes
            else:
                y_hat[i] = self.y_hat_no

        return y_hat

Next we will make minor modifications to create a random stump

In [301]:
class RandomStump(DecisionStump):
    def fit(self, X, y):
        d = X.shape[1]
        sqrt_d = int(np.floor(np.sqrt(d)))

        chosen_features = np.random.choice(d, sqrt_d, replace=False)

        DecisionStump.fit(self, X, y, feature_space=chosen_features)

## Creating the Decision Tree
We will now create a decision tree by recursively splitting at each stump

In [302]:
class DecisionTree:
    stump = None
    yes_path = None
    no_path = None

    def __init__(self, max_depth, random=False):
        self.max_depth = max_depth
        self.random = random

    def fit(self, X, y):
        stump = RandomStump() if self.random else DecisionStump()
        stump.fit(X, y)

        if self.max_depth <= 1 or stump.j_best is None: # max depth leaf stump or no point in splitting
            self.stump = stump
            self.yes_path = None
            self.no_path = None
            return

        j = stump.j_best
        value = stump.t_best

        yes = X[:, j] > value
        no = X[:, j] <= value

        # recursively build tree
        self.stump = stump
        self.yes_path = DecisionTree(self.max_depth - 1)
        self.yes_path.fit(X[yes], y[yes])
        self.no_path = DecisionTree(self.max_depth - 1)
        self.no_path.fit(X[no], y[no])

    def predict(self, X):
        n, d = X.shape
        y = np.zeros(n)


        if self.stump.j_best is None or self.yes_path is None:  # just use this stump
            y = self.stump.predict(X)
        else:
            j = self.stump.j_best
            t = self.stump.t_best

            yes = X[:, j] > t
            no = X[:, j] <= t

            y[yes] = self.yes_path.predict(X[yes])
            y[no] = self.no_path.predict(X[no])

        return y

# Utilizing the Decision Tree
Let us see what validation error we can achieve with a single decision tree with various depths while making use of k fold validation with k = 4

In [303]:
depths = [5,7,10]

def k_fold_validation(model, X, y, k=4):
    total_accuracy = 0
    for i in range (1,k+1):
        left = int(len(X)*((i-1)*(1/k)))
        right = int(len(X)*((i)*(1/k)))
        X_fold_train = np.concatenate([X[:left],X[right:]])
        X_fold_test = X[left:right]
        y_fold_train = np.concatenate([y[:left],y[right:]])
        y_fold_test = y[left:right]

        model.fit(X_fold_train, y_fold_train)
        y_hat_test = model.predict(X_fold_test)
        err_test = np.mean(y_hat_test != y_fold_test)
        accuracy = 1 - err_test
        total_accuracy += accuracy
    
    return total_accuracy / k

for depth in depths:
    model = DecisionTree(depth)
    validation_acc = k_fold_validation(model, X_train, y_train, k=3)
    print(f"depth:{depth}, validation accuracy: {validation_acc}")

c:\Users\brian\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\stats\_entropy.py:135: RuntimeWarning: invalid value encountered in divide
  pk = 1.0*pk / np.sum(pk, axis=axis, keepdims=True)


depth:5, validation accuracy: 0.8054984135839057
depth:7, validation accuracy: 0.7962487248498782
depth:10, validation accuracy: 0.7799987853035223


As seen above increasing depth from 5 to 10 increases our validation accuracy but from 10 to 15 it drops likely due to overfitting with increased depth. Regardless, it is at a very slow rate and we are still getting quite a poor prediction so lets move on

## Random Tree
We will generate a random tree which differs from the decision tree in that it bootstraps the passed in X and y and the stumps will only be applied to $\sqrt{d}$ features. This can simply be done with a few modifications to our original tree and stump

In [304]:
class RandomTree(DecisionTree):
    def __init__(self, max_depth):
        DecisionTree.__init__(self, max_depth=max_depth, random=True)

    def fit(self, X, y):
        n = X.shape[0]
        boostrap_indices = np.random.choice(n, n, replace=True)
        bootstrap_X = X[boostrap_indices]
        bootstrap_y = y[boostrap_indices]

        DecisionTree.fit(self, bootstrap_X, bootstrap_y)


class RandomForest:
    num_trees = 0
    max_depth = 0
    trees = []

    def __init__(self, num_trees, max_depth):
        self.num_trees = num_trees
        self.max_depth = max_depth

    def fit(self, X, y):
        for i in range(self.num_trees):
            tree = RandomTree(self.max_depth)
            tree.fit(X, y)
            self.trees.append(tree)

    def predict(self, X):
        y_pred = None
        for i in range(len(self.trees)):
            curr_pred = self.trees[i].predict(X)
            if(i == 0):
                y_pred = curr_pred
            else: 
                np.vstack((y_pred, curr_pred))
        
        return mode(y_pred)[0].flatten()

Now let us create a random forest with 5 random trees see our results

In [305]:
random_forest = RandomForest(5, 10)
validation_acc = k_fold_validation(random_forest, X_train, y_train, k=4)
print(f"validation accuracy: {validation_acc}")

C:\Users\brian\AppData\Local\Temp\ipykernel_26388\2441632914.py:38: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return mode(y_pred)[0].flatten()


In [ ]:
random_forest = RandomForest(5, 10)
validation_acc = k_fold_validation(random_forest, X_train, y_train, k=4)
print(f"validation accuracy: {validation_acc}")

C:\Users\brian\AppData\Local\Temp\ipykernel_26388\2441632914.py:38: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return mode(y_pred)[0].flatten()


validation accuracy: 0.28583333333333333


In [ ]:
from sklearn.naive_bayes import GaussianNB

forest = ensemble.RandomForestClassifier(n_estimators=5, max_depth=5)
forest.fit(X_train,y_train)
lr = linear_model.LinearRegression()
lr.fit(X_train,y_train)
y_hat_lr = lr.predict(X_train)
y_hat = forest.predict(X_train)
print(np.count_nonzero(y_train == y_hat)/len(y_hat))
print(np.count_nonzero(y_train == y_hat_lr))

y_test_hat = forest.predict(X_test)
print(np.count_nonzero(y_test == y_test_hat)/len(y_test))

nb = GaussianNB()
nb.fit(X_train, y_train)
y_hat = nb.predict(X_test)
print(np.count_nonzero(y_train == y_hat)/len(y_hat))
print(np.count_nonzero(y_test == y_test_hat)/len(y_test))

print(len(y_train[y_train == 1])/len(y_train))

0.45
0
0.415
0.0
0.415
0.28583333333333333


C:\Users\brian\AppData\Local\Temp\ipykernel_26388\1647491162.py:18: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  print(np.count_nonzero(y_train == y_hat)/len(y_hat))
